In [ ]:
# import libraries
import requests
from PIL import Image

# suppress warnings
import warnings
warnings.filterwarnings("ignore")

try:
    import gradio as gr
    from together import Together

except ImportError:
    !pip install -q together
    !pip install -q gradio==5.17.1

import gradio as gr
from together import Together

# Get Client
your_api_key = "x"
client = Together(api_key=your_api_key)

In [ ]:
def respond(user_message, chat_history):
    # 1. Add your external knowledge here
    external_knowledge = """
    Brazil is a vibrant country known for its carnival, samba, soccer, and beautiful landscapes.
    It is the largest country in South America and home to the Amazon rainforest.
    Brazil's culture is a mix of influences and many other cultures.
    """

    # 2. Generate a text response using the llm
    chatbot_prompt = f"""
    You know how to play a fun and easy game about Brazil!

    Respond to this "{user_message}" following these instructions:

    ## Instructions:
    * Keep your answers short and friendly.
    * Always start with "Ahhhhhhh!"
    * Add a playful tone to every response.
    * Include a fun fact about Brazil when possible.
    * Base all your answers on this: {external_knowledge}
    """

    response = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3-8B-Instruct-Lite",
        messages=[{"role": "user", "content": chatbot_prompt}],
    )
    response_text = response.choices[0].message.content

    # 3. Generate image based on the response
    def gen_image(prompt, width=256, height=256):
        response = client.images.generate(
            prompt=prompt,
            model="stabilityai/stable-diffusion-xl-base-1.0",
            steps=30,
            n=1,
        )
        return response.data[0].url

    image_prompt = f"A funny cartoon of {response_text} in a pop art style"
    image_url = gen_image(image_prompt)

    # 4. Append the response and image to the chat history
    chat_history.append((user_message, response_text))
    return "", chat_history, image_url

# 5. Create the chatbot interface
with gr.Blocks(theme=gr.themes.Soft()) as app:

    # Add Brazilian flag and title
    gr.HTML(f"""
    <div style="text-align: center;">
        <img src="https://upload.wikimedia.org/wikipedia/commons/0/05/Flag_of_Brazil.svg" alt="Brazilian Flag" width="100" style="margin-right: 20px;">
        <h2 style="color: {green}; display: inline-block;">AI Chatbot</h2>
    </div>
    """)

    gr.Markdown(f"""
    <div style="text-align: center; font-size: 1.2em; color: {green};">
        Welcome! Type your message below and enjoy a Brazilian-inspired chat experience!
    </div>
    """)

    chatbot = gr.Chatbot(label="Chat History", elem_id="chatbot", show_label=True)
    image_output = gr.Image(label="Generated Image", elem_id="image_output")

    user_input = gr.Textbox(
        placeholder="Type your message here...",
        label="Your Message",
        elem_id="user_input"
    )

    send_button = gr.Button("Send ", elem_id="send_button")

    send_button.click(
        respond,
        inputs=[user_input, chatbot],
        outputs=[user_input, chatbot, image_output],
    )

    user_input.submit(
        respond,
        inputs=[user_input, chatbot],
        outputs=[user_input, chatbot, image_output],
    )

app.launch()
